### 11725 : 트리의 부모 찾기
- 1번 노드가 루트 노드라고 주어짐
- 부모 노드의 인덱스를 저장하는 배열을 만들고, DFS를 하면서 첫 방문일 때 위 배열에서 부모 노드를 저장하면 됨

In [12]:
import sys
sys.setrecursionlimit(int(1e6))
sys.stdin = open('test.txt', 'r')
input = sys.stdin.readline

N = int(input())
graph = [[] for _ in range(N + 1)]

for _ in range(N - 1):
    a, b = map(int, input().split())
    graph[a].append(b)
    graph[b].append(a)

visited = [0] * (N + 1)
parent = [0] * (N + 1)

def dfs(start_node):
    visited[start_node] = 1
    for i in graph[start_node]:
        if visited[i] == 0:
            parent[i] = start_node
            dfs(i)

dfs(1)
for i in range(2, N + 1):
    print(parent[i])

1
1
2
3
3
4
4
5
5
6
6


### 1167 : 트리의 지름
- 트리의 지름 찾는 알고리즘
    1. 아무 점이나에서 시작, 가장 먼 점을 찾는다(DFS, 방문 리스트에 값을 누적하면 됨)
    2. 그 가장 먼 점에서 다시 가장 먼 점을 찾음
    3. 2에서 찾은 거리가 트리의 지름임
    
- 다음 문제(1967)는 같은 문제니까 스킵

In [20]:
import sys
sys.setrecursionlimit(int(1e6))
sys.stdin = open('test.txt', 'r')
input = sys.stdin.readline

N = int(input())
graph = [[] for _ in range(N+1)]

def dfs(start_node):
    """방문 리스트를 구현할 때 시작 노드에서의 거리 값을 저장함"""
    for next_node, weight in graph[start_node]:
        if visited[next_node] == -1:
            visited[next_node] = visited[start_node] + weight
            dfs(next_node)
    
    
for _ in range(N):
    arr = list(map(int, input().split()))
    x = arr[0]
    # arr의 원소 개수는 간선의 갯수 * 2임
    for i in range(1, len(arr), 2):
        if arr[i] == -1:
            break
        graph[x].append((arr[i], arr[i+1])) # 연결된 노드, 가중치


visited = [-1] * (N + 1)

start_node = 1
visited[start_node] = 0
dfs(start_node) # ㄹㅇ 아무거나

# visited에서 나온 최대 거리의 인덱스를 얻는다
max_dist_node = visited.index(max(visited))

# 다시 방문 배열을 초기화한 다음
visited = [-1] * (N + 1)
visited[max_dist_node] = 0
dfs(max_dist_node)

# 이 방문 배열의 가장 큰 값이 트리의 지름이다
print(max(visited))

11


### 1991 : 트리 순회
- 이진 트리를 입력 받았을 때 전위, 중위, 후위 순회 결과를 출력

In [31]:
import sys
sys.setrecursionlimit(int(1e6))
sys.stdin = open('test.txt', 'r')
input = sys.stdin.readline

N = int(input())
graph = {}
for _ in range(N):
    parent, left, right = input().strip().split()
    graph[parent] = [left, right]
    
def preorder(node):
    print(node, end = '')
    left, right = graph[node][0], graph[node][1]
    if left != '.':
        preorder(left)
    if right != '.':
        preorder(right)

def inorder(node):
    left, right = graph[node][0], graph[node][1]
    if left != '.':
        inorder(left)
    print(node, end = '')
    if right != '.':
        inorder(right)

def postorder(node):
    left, right = graph[node][0], graph[node][1]
    if left != '.':
        postorder(left)
    if right != '.':
        postorder(right)
    print(node, end = '')

preorder('A')
print()
inorder('A')
print()
postorder('A')

ABDCEFG
DBAECFG
DBEGFCA

### 2263 : 중위, 후위 순회가 주어졌을 때 전위 순회 구하기
- 제출 : pypy3
1. 후위 순회의 마지막에 있는 노드는 무조건 루트 노드이다.
2. 이 루트 노드의 위치를 중위 순회에서 찾을 수 있다면, 왼쪽 서브트리와 오른쪽 서브 트리로 나눌 수 있다
3. 중위 순회든 후위 순회든 왼쪽 서브 트리에 대한 탐색을 먼저 한다.
    - 따라서 후위 순회에서도 어디까지가 왼쪽 서브트리이고 어디까지가 오른쪽 서브트리인지 알 수 있다
4. 이런 식으로 계속 쪼개나가면 배열 길이가 1이 될 때까지 쪼갤 수 있는데, 각 단계에서 루트 노드를 찾아서 출력 -> 왼쪽 서브 트리 출력 -> 오른쪽 서브 트리 출력하는 과정을 반복하면 될 듯?

In [41]:
import sys
sys.setrecursionlimit(int(1e5)) # 이 문제 특수임
sys.stdin = open('test.txt', 'r')
input = sys.stdin.readline

N = int(input())
inorder = list(map(int, input().split()))
postorder = list(map(int, input().split()))

def func(inorder_start, inorder_end, postorder_start, postorder_end):
    """각 인풋은 인덱스를 의미함"""
    
    # 분할 정복의 종결조건은 그냥 이걸로 외우면 됨
    if inorder_start > inorder_end or postorder_start > postorder_end:
        return
    
    root = postorder[postorder_end]
    mid = inorder.index(root)
    
    # 서브트리의 길이
    left = mid - inorder_start # a~b의 길이는 b-a+1이지만, b는 포함X
    right = inorder_end - mid # 마찬가지
    
    # 출력 : 전위순회
    print(root, end=" ")

    """
    인덱스 지정 관련
    inorder는 mid를 기준으로 왼쪽 오른쪽으로 나누면 됨
    postorder를 생각해보면, inorder에서 서브트리의 길이를 얻을 수 있음
    왼쪽 서브트리 : 즉 이전에 갖고 있던 postorder의 시작점이 있고
                    inorder에서 왼쪽 서브트리의 길이를 안다면
                    왼쪽 서브트리의 인덱스는 postorder ~ postorder + left - 1까지임
                    (1을 빼는 건 두 점을 포함한 길이는 마지막에 1을 빼니까)
    오른쪽 서브트리 : inorder에서 구한 오른쪽 서브트리의 길이가 있다면
                     postorder의 끝점 - 오른쪽 서브트리의 길이가 시작점임
                    (마지막점이 루트이기 때문에 1을 더하지 않음)
                    마찬가지로 오른쪽 서브트리의 마지막 점은 기존 끝점 - 1
    """
    func(inorder_start, mid - 1, postorder_start, postorder_start + left - 1)
    func(mid + 1, inorder_end, postorder_end - right, postorder_end - 1)
    
func(0, N-1, 0, N-1)

2 1 3 

### 5639 : 이진 검색 트리
- 이진 검색 트리를 전위순회한 결과가 있을 때 후위 순회 결과를 구하시오

In [67]:
import sys
sys.setrecursionlimit(int(1e6)) 
sys.stdin = open('test.txt', 'r')
input = sys.stdin.readline

preorder = []
while True:
    inp = input()
    if inp == '':
        break
    preorder.append(int(inp))

N = len(preorder)
    
def func(start, end):
    root = preorder[start]
    
    # if start == end:
    #     print(root)
    
    """
    서브트리를 가지지 않는 조건
    - 왼쪽 서브트리 : 나머지 값들이 루트보다 모두 크면
                    = 분할점이 preorder의 가장 왼쪽에 있다면
    - 오른쪽 서브트리 : 나머지 값들이 루트보다 모두 작으면
                    = 분할점이 preorder의 가장 오른쪽에 있다면
    """
    left_end = start
    for i in range(start + 1, end + 1):
        # 여기 조건도 신경 쓰자 : 
        # 루트보다 작은 가장 끝 점을 찾으려면
        # 작은 값이 계속 갱신되어야 함
        if preorder[i] < preorder[start]:
            left_end = i
    
    if left_end != start:
        func(start + 1, left_end)
    if left_end != end:
        func(left_end + 1, end)
    print(root)

func(0, N - 1)

5
28
24
45
30
60
52
98
50


#### preorder
- 루트노드가 맨 앞에 나온다
- 이건 이진 탐색 트리이니까, 오른쪽에는 루트노드보다 큰 값이 나온다
- 그러면 왼쪽 서브트리의 마지막 지점을 찾을 수 있다 : 루트 노드보다 작으면 됨
- 좌우로 서브 트리를 나눌 수 있다..
- 반복하면 되겠네?

### 4803 : 트리 판별


In [77]:
import sys
sys.setrecursionlimit(int(1e6)) 
sys.stdin = open('test.txt', 'r')
input = sys.stdin.readline

def dfs(start_node, parent_node):
    
    # 초기 parent_node는 0을 쓰면 됨
    
    visited[start_node] = 1
    
    for i in graph[start_node]:
        
        if i == parent_node:
            continue
            
        elif visited[i]:
            return False
        
        # 방문한 적 없고, 부모 노드가 아니면 재귀임
        # 밑에서 False가 나왔다면 끌어올려줘야 함
        if dfs(i, start_node) == False:
            return False

    return True

case = 1

while True:
    
    N, M = map(int, input().split())
    if N == 0 and M == 0:
        break
        
    graph = [[] for _ in range(N + 1)]
    for _ in range(M):
        a, b = map(int, input().split())
        graph[a].append(b)
        graph[b].append(a)
        
    visited = [0] * (N + 1)
    ans = 0
    
    for i in range(1, N + 1):
        if visited[i] == 0:
            ans += dfs(i, 0)
    
    if ans == 0:
        print(f"Case {case}: No trees.") # 아 또 이거 대문자로 씀;
    elif ans == 1:
        print(f"Case {case}: There is one tree.")
    else:
        print(f"Case {case}: A forest of {ans} trees.")
        
    case += 1

Case 1: A forest of 3 trees.
Case 2: There is one tree.
Case 3: No Trees.


- 트리 판별 조건식은
    - `다음에 방문할 노드가 직전 노드가 아니고 방문한 적 있는 노드라면 트리가 아니다` 정도만 생각난다
    - 근데 이 문제는 닿지 않는 노드도 있음
    
- 그러면 
1. 각 점에 대해 dfs를 돌림
    - 방문한 적 있는 노드라면 걍 무시
2. 그러면 그 점과 연결된 노드들을 모두 방문하게 됨 
    - 이 때 트리를 판별함 : 직전 노드가 아닌데 방문한 적이 있다면 사이클이 있기 때문에 트리가 아님
    - 1개의 노드만 있어도 트리로 간주되니까, 기본 값을 True로 놓음